In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [ ]:
mongo_client = MongoClient(os.environ["MONGO_CONNECTION_STRING"])

In [ ]:
user_db = mongo_client["users"]["user-data"]

In [ ]:
cursor = user_db.find({})

In [ ]:
for user in cursor:
    _id = user["company_id"]
    new_moments = {
        "general_news":["A", "bf", "XC"],
        "industry_news":["A", "bfa", "XC"],
        "current_events":["A", "basd", "XC"],
        "social_media":["A", "b", "XCsad"],
    }

    new_values = {"$set": {"moments": new_moments}}
    print(user_db.update_one({"company_id":_id}, new_values))


In [ ]:
from chroma_interface.reader import Reader
from chroma_interface.writer import Writer
import os

keys = list(open(".env", "r"))

for item in keys:
    variable, value = item.split("=")[0], "=".join(item.split("=")[1:])
    os.environ[variable] = value.replace("\n", "")

In [ ]:
openai_key = os.environ["OPENAI_API_KEY"]
chroma_host = os.environ["CHROMA_IP"]

reader = Reader(host=chroma_host, port=8000, collection="test-1", openai_api_key=openai_key)
writer = Writer(host=chroma_host, port=8000, openai_api_key=openai_key)

In [ ]:
collections = reader.list_collections()

In [1]:
from utils.simple_utils import *
from utils.google_utils import *
from utils.langchain_utils import *

from mongo.interface import *

from chroma_interface import *
from chroma_interface.reader import Reader
from chroma_interface.writer import Writer

import os
import time
# Constants
NEWS_TOPICS = read_lines_from_file("./config/topic-type.txt")
QUERY_TOPICS = read_lines_from_file("./config/query-topics.txt")
COUNTRY_NAMES = read_lines_from_file("./config/countries-names.txt")
COUNTRY_CODES = defaultdict(None)
for name, code in zip(COUNTRY_NAMES, read_lines_from_file("./config/countries-code.txt")):
    COUNTRY_CODES[name] = code


# environment setup
with open(".env", "r") as key_file:
    keys = list(key_file)

for item in keys:
    variable, value = item.split("=")[0], "=".join(item.split("=")[1:])
    os.environ[variable] = value.replace("\n", "")

topic=NEWS_TOPICS[0]
country_name = "India"
country_code=COUNTRY_CODES[country_name]
lang='en'
collection_name = "general_news"

chroma_reader, chroma_writer = get_reader_writer(
    host=os.environ["CHROMA_IP"],
    port=int(os.environ["CHROMA_PORT"]),
    openai_api_key=os.environ["OPENAI_API_KEY"],
    reader_collection_name=collection_name
)

user_mongo_client = MongoInterface(
    url=os.environ["MONGO_CONNECTION_STRING"],
    database="users",
    collection="user-data"
)

from scripts import *


In [2]:
collection_name="current_events"
create_new_collection(chroma_reader, chroma_writer, collection_name)


Creating Collection: current_events


In [9]:
news_df = pd.DataFrame()

news_df = pd.concat([
                news_df, 
                get_news_by_search(f"upcoming or ongoing major events about {topic}", limit=50, country=country_code)
            ])

Getting headlines for upcoming or ongoing major events about WORLD:  41%|████      | 21/51 [00:33<00:44,  1.49s/it]c:\Users\vedan\AppData\Local\Programs\Python\Python310\lib\site-packages\dateutil\parser\_parser.py:1207: UnknownTimezoneWarning: tzname CST identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
Getting headlines for upcoming or ongoing major events about WORLD: 100%|██████████| 51/51 [01:27<00:00,  1.72s/it]


In [11]:
news_df = news_df[(news_df["description"] != "")]
news_df["country_code"] = [country_code] * len(news_df)

In [2]:
get_current_events()

Creating Collection: current_events


Getting headlines for upcoming or ongoing major events about entertainment: 100%|██████████| 51/51 [01:06<00:00,  1.30s/it]


InvalidRequestError: '$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.

In [ ]:
collection_name = "current_events"

chroma_reader.set_collection(collection_name)

current_events_moments = generate_current_events(chroma_reader,QUERY_TOPICS)

In [ ]:
current_events_moments

In [ ]:
googleSearch(f"upcoming or ongoing major events about sports tournaments", num_results=20, country=country_code)

In [ ]:
get_news_by_search(f"upcoming or ongoing major events about sports tournaments", country=country_code, limit=20)

In [ ]:
chroma_reader.search(f"List 10 important and interesting events related to any of these topics: {','.join(QUERY_TOPICS)}.", n=30, filter={"country_code":"IN"})

In [ ]:
relevant_docs

In [ ]:
current_events_moments

In [ ]:
get_metadata(article)

In [ ]:
temp_links = [('https://www.visitphilly.com/articles/philadelphia/top-events-and-festivals-in-philadelphia/', 'IN'), ('https://en.wikipedia.org/wiki/News', 'IN'), ('https://www.nih.gov/about-nih/what-we-do/nih-almanac/national-institute-drug-abuse-nida', 'IN'), ('https://www.cnn.com/', 'IN'), ('https://blog.twitter.com/en_us/topics/insights/2022/how-many-people-come-twitter-for-news', 'IN')]
articles_data = [parse_for_current_events(item) for item in temp_links]

In [ ]:
target_key="source"

In [ ]:
docs, metadata = build_splited_docs(articles_data, target_key="source")

In [ ]:
metadata

In [ ]:
page = articles_data[0]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ".", ",", "\n"])


In [ ]:
splits = text_splitter.split_text(page[target_key])


In [ ]:
page.pop(target_key, None)

In [ ]:
page

In [ ]:
get_current_events()

In [ ]:
create_new_collection(chroma_reader, chroma_writer, collection_name)

In [ ]:
def build_splited_docs(sitetexts):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0, separators=[" ", ".", ",", "\n"])

    docs, metadatas = [], []
    for page in sitetexts:
        splits = text_splitter.split_text(page['text'])
        docs.extend(splits)
        metadatas.extend([{"source": page['source'], "country_code": page["country_code"]}] * len(splits))

    return docs, metadatas

In [ ]:
import pickle as pkl

In [ ]:
(docs, metadata) = pkl.load(open("test.pkl", "rb"))

In [ ]:
chunked_docs = divide_chunks(docs, 1000)
chunked_metadata = divide_chunks(metadata, 1000)

In [ ]:
for docs, metadata in zip(chunked_docs, chunked_metadata):
    chroma_writer.update("test_industry", docs, metadata)

In [ ]:
chroma_writer.delete_collection("general_news")

In [ ]:
chroma_reader.list_collections()

In [ ]:
chroma_reader.set_collection("test_industry")

In [ ]:
outputs = chroma_reader.search("Indian Food", n=40)

In [ ]:
news_df = get_news_by_topic("SPORTS", country=country_code, lang=lang)

In [ ]:
news_df = news_df[(news_df["top_image"] != "") & (news_df["keywords"] != "")]


In [ ]:
article = parse_news_url(news_df["url"][0])

In [ ]:
news_df.iloc[0]

In [ ]:
news_df["keywords"][0]

In [ ]:
article.meta_keywords

In [ ]:
def load_df(df, page_content_column="title"):
    data = DataFrameLoader(df, page_content_column=page_content_column).load()

    documents = [item.page_content for item in data]
    metadata = [item.metadata for item in data]

    return documents, metadata

In [ ]:
documents, metadata = load_df(news_df)

In [ ]:
chroma_writer.update("general_news", documents, metadata)

In [ ]:
article = parse_news_url("https://news.google.com/rss/articles/CBMic2h0dHBzOi8vd3d3Lndoby5pbnQvbmV3cy9pdGVtLzEwLTExLTIwMjMtY2hpbGRyZW4td2l0aC1jYW5jZXItZXZhY3VhdGVkLWZyb20tZ2F6YS1mb3ItdHJlYXRtZW50LXRvLWVneXB0LWFuZC1qb3JkYW7SAQA?oc=5")

In [ ]:
outputs = chroma_reader.search("Indian Food")[0]

In [ ]:
chroma_reader.search("Indian Food")

In [ ]:
chroma_reader.set_collection("general_news")

In [ ]:
filtered = filter_news("Diwali", chroma_reader)

In [ ]:
general = filtered[:5]
industry = filtered[5:11]

In [ ]:
user_mongo_client.update_user_moments(
    100,
    {
        "general_news": general,
        "industry": industry,
        "current_events": [
            {
                "event_name": "ICC Cricket world cup 2023",
                "topic": "Cricket world cup hosted in India",
                "validation": {
                    "google_trends": {
                        "keywords": ["cricket", "icc world cup", "india cricket"]
                    }
                }
            },
            {
                "event_name": "Diwali",
                "topic": "One of the biggest festivals in India",
                "validation": {
                    "google_trends": {
                        "keywords": ["diwali", "indian festivals", "lights"]
                    }
                }
            }
        ],
        "social_media": [
            {
                "title": "Run the dog trends",
                "hashtag": ["#crazydog", "#mrdoge", "#woof"],
                'validation': {
                    "hashtags": [
                        {"#crazydog": 100210,
                         "#mrdoge": 100232, 
                         "#woof": 100311}
                    ]
                }
            },
            {
                "title": "Night is day? what?",
                "hashtag": ["#idontunderstand", "#oogaboogabrain", "#what"],
                'validation': {
                    "hashtags": [
                        {"#idontunderstand": 100210,
                         "#oogaboogabrain": 100232, 
                         "#what": 100311}
                    ]
                }
            }
        ]
    }
)